In [1]:
! pip3 install pycodestyle flake8 pycodestyle_magic

In [2]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [3]:
from pprint import pprint

### 2. обработаем книгу с помощью mystem:

In [4]:
import json
from pymystem3 import Mystem

In [5]:
m = Mystem()

In [6]:
with open("yoga-sna.txt_Ascii.txt", encoding="utf-8") as f:
    text = f.read()

#### распарсим и замерим время работы:

In [7]:
%%time

anad_text = m.analyze(text)

CPU times: user 137 ms, sys: 32.9 ms, total: 170 ms
Wall time: 2.79 s


#### сохраним результат работы в .json

In [8]:
with open('yoga-sna_mystem.json', 'w', encoding='utf-8') as f:
    json.dump(anad_text, f, ensure_ascii=False)

### 3. обработаем книгу с помощью pymorphy:

In [9]:
import nltk
from nltk.tokenize import word_tokenize

#### токенизируем текст с помощью nltk и замерим время работы:

In [10]:
%%time


tokens = word_tokenize(text)

#### разберем слова с помощью pymorphy и замерим время работы::

In [11]:
from pymorphy2 import MorphAnalyzer

In [12]:
morph = MorphAnalyzer()

In [13]:
%%time

pymd_text = []

for token in tokens:
    pym = morph.parse(token)
    pymd_text.append(pym)

#### создаем список с леммами и их частью речи:

In [14]:
lemmd_list = []
for item in pymd_text:
    unit = item[0]
    lemmd_list.append([unit.normal_form, unit.tag.POS])

#### сохраним результат работы в .json

In [15]:
with open('yoga-sna_pymorphy.json', 'w', encoding='utf-8') as f:
    json.dump(lemmd_list, f, ensure_ascii=False)

### 4. ответим на вопросы:

#### какую долю слов составляет каждая часть речи?

In [16]:
from collections import Counter

#### выведим леммы и их часть речи, посчитаем последние

In [17]:
part = []
for i in lemmd_list:
    part.append(i[1])
total = Counter(part)
parts = list(total.items())
for i in range(len(parts)):
    print(parts[i][0], '-', parts[i][1])

In [18]:
summa = len(text)
for i in range(len(parts)):
    print(parts[i][0], 'составляет ', parts[i][1]/summa, 'долю')

#### найдем топ-20 глаголов:

In [19]:
verb = []
for unit in lemmd_list:
    if unit[1] == 'VERB':
        verb.append(unit[0])
verb_total = Counter(verb)
verbs = verb_total.most_common(20)
for i in range(len(verbs)):
    print(verbs[i][0], '-', verbs[i][1])

#### найдем топ-20 наречий:

In [20]:
adverb = []
for unit in lemmd_list:
    if unit[1] == 'ADVB':
        adverb.append(unit[0])
adverb_total = Counter(adverb)
adverbs = adverb_total.most_common(20)
for i in range(len(adverbs)):
    print(adverbs[i][0], '-', adverbs[i][1])

### 5. топ-25 биграмм и триграмм:

#### найдем леммы:

In [21]:
lemmas = []
for i in lemmd_list:
    lemmas.append(i[0])

#### почистим:

In [22]:
punct = """!"#$%&\'()*+,./:;<=>?@[\\]^_-`{|}~«»"""
for char in punct:
    for i in lemmas:
        if char == i:
            lemmas.remove(char)

#### найдем топ-25 биграмм:

In [23]:
bigrams = nltk.bigrams(lemmas)
bis = []
for b in bigrams:
    bis.append(b)
bi_total = Counter(bis)
bi = bi_total.most_common(25)
for i in range(len(bi)):
    print(bi[i][0], '-', bi[i][1])

#### найдем топ-25 триграмм:

In [24]:
trigrams = nltk.trigrams(lemmas)
tris = []
for t in trigrams:
    tris.append(t)
tri_total = Counter(tris)
tri = tri_total.most_common(25)
for i in range(len(tri)):
    print(tri[i][0], '-', tri[i][1])